In [1]:
# 1. Imports
import pymc as pm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

# 2. Load dataset
housing = fetch_california_housing()
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df["MedHouseVal"] = housing.target

# 3. Use AveRooms to predict MedHouseVal
X = df["AveRooms"].values
y = df["MedHouseVal"].values

# 4. Standardize the predictor for numerical stability
X_mean, X_std = X.mean(), X.std()
X_scaled = (X - X_mean) / X_std

# 5. Build Bayesian Regression Model
with pm.Model() as model:
    # Priors
    intercept = pm.Normal("intercept", mu=0, sigma=10)
    slope = pm.Normal("slope", mu=0, sigma=10)
    sigma = pm.HalfNormal("sigma", sigma=10)

    # Linear model
    mu = intercept + slope * X_scaled

    # Likelihood
    y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=y)

    # Sampling
    trace = pm.sample(1000, tune=1000, return_inferencedata=True, target_accept=0.95)

az.plot_posterior(trace, var_names=["intercept", "slope", "sigma"], hdi_prob=0.95)
plt.show()

# Sample from posterior
posterior_samples = trace.posterior
slope_samples = posterior_samples["slope"].values.flatten()
intercept_samples = posterior_samples["intercept"].values.flatten()

# Predict across X range
x_vals = np.linspace(X_scaled.min(), X_scaled.max(), 100)
y_preds = np.array([
    intercept_samples[i] + slope_samples[i] * x_vals for i in range(100)
])

# Plot
plt.figure(figsize=(10, 6))
plt.scatter(X_scaled, y, alpha=0.4, label="Data")
plt.plot(x_vals, y_preds.mean(axis=0), color="red", label="Mean Prediction")
plt.fill_between(
    x_vals,
    np.percentile(y_preds, 2.5, axis=0),
    np.percentile(y_preds, 97.5, axis=0),
    alpha=0.3,
    color="red",
    label="95% CI"
)
plt.xlabel("AveRooms (Standardized)")
plt.ylabel("Median House Value ($100,000s)")
plt.title("Bayesian Linear Regression (California Housing)")
plt.legend()
plt.show()



ModuleNotFoundError: No module named 'pymc'